In [ ]:
import pandas as pd
import numpy as np


In [ ]:
df_date_temperature = pd.read_csv('/Salmon_project/data/FW_temperature_filled.csv')
df_date_cycles = pd.read_csv('Salmon_project/data/FW_cycle_dates.csv')
df_strain_season  = pd.read_csv('Salmon_project/data/smolt_dataset.csv')
datatypes_1 = df_date_temperature.dtypes
datatypes_2 = df_date_cycles.dtypes
df_date_temperature = df_date_temperature.dropna()
df_date_temperature ['event_date']= pd.to_datetime(df_date_temperature ['event_date'])
df_date_cycles [['first_movement_date', 'first_feeding_date', 'shipout_date']] = df_date_cycles [['first_movement_date', 'first_feeding_date', 'shipout_date']].apply(pd.to_datetime)
df_date_cycles = df_date_cycles.reindex(columns = df_date_cycles.columns.tolist()+['days_in_fresh_water','average_tempr','median_tempr', 'strain', 'season'])
n = 0
while n < len(df_date_cycles):
    df_date_cycles.at[n, 'days_in_fresh_water'] = int((df_date_cycles.at[n, 'shipout_date'] - df_date_cycles.at[n, 'first_movement_date'])/np.timedelta64(1, 'D'))
    part_number = df_date_cycles.at[n, 'pretransfer_fw_locus_population_id']
    date_start_feeding = df_date_cycles.at[n, 'first_feeding_date']
    date_shipout = df_date_cycles.at[n, 'shipout_date']
    filtered_df = df_date_temperature[(df_date_temperature['event_date']>= date_start_feeding) & (df_date_temperature['event_date'] <= date_shipout) & (df_date_temperature['final_locus_population_id']== part_number)]
    df_date_cycles.at[n, 'average_tempr'] = filtered_df['temperature'].mean()
    df_date_cycles.at[n, 'median_tempr'] = filtered_df['temperature'].median()
    filtered_df_2 = df_strain_season[(df_strain_season['from_locus_population_id'] == part_number)]
    #strain = filtered_df_2['strain']
    print(filtered_df_2)
    n = n + 1
    #print (n, date_start_feeding, date_shipout, part_number)
#writer = pd.ExcelWriter('/Users/evgenijnotebook/Desktop/Salmon_project/df_common_data .xlsx')
#df_date_cycles.to_excel(writer)
#writer.save()

In [ ]:
#;df_date_temperature = pd.read_csv('Salmon_project/data/FW_temperature_filled_test.csv')
#df_date_temperature = df_date_temperature.dropna()
#part_number_list = df_date_temperature['final_locus_population_id'].tolist()
#unic_part_number_list = []
#df_temperature_cleared = pd.DataFrame(columns = ['final_locus_population_id','event_date','temperature_cleared'])
#for item in part_number_list:
##    if item not in unic_part_number_list:
#        unic_part_number_list.append(item)
#n = 0
#while n < len(unic_part_number_list):
#    part_number = unic_part_number_list[n]
#   df_temperature_filtered = df_date_temperature[(df_date_temperature['final_locus_population_id'] == part_number)]
#    #df_temperanure_filtered = df_temperature_filtered.reindex(columns = df_temperature_filtered.columns.tolist()+['rolling_tempr','tempr_cleared'])
#    df_temperature_filtered['rolling_tempr'] = df_temperature_filtered['temperature'].rolling(30).mean()
#    df_temporary = df_temperature_filtered.head(30)
#    average_tempr_30 = df_temporary['temperature'].mean()
#    half_average_tempr_30 = average_tempr_30 / 2 
#    k = 0
#    while k < len(df_temporary):
#        t_1 = df_temperature_filtered.at[k,'temperature']
#        t_dif = t_1 - average_tempr_30
#        print ('t_1', t_1, 't_dif', t_dif)
#        if abs(t_dif) < half_average_tempr_30:
#                print ('yes')
#                df_temperature_filtered.at[k,'rolling_tempr'] = t_1
#        else:
#                print('not')
#                df_temperature_filtered.at[k,'rolling_tempr'] = average_tempr_30   
#        
#        print('k', k )
#        k = k + 1        
#    df_temperature_cleared = df_temperature_cleared.append(df_temperature_filtered)        
#    print('n', n)
#    n = n + 1
#    #df_temperature_filtered.drop(columns= 'rolling_tempr', inplace=True)
   

In [ ]:
df_date_temperature = pd.read_csv('Salmon_project/data/FW_temperature_filled_test_2.csv')
df_date_temperature = df_date_temperature.dropna()
part_number_list = df_date_temperature['final_locus_population_id'].tolist()
unic_part_number_list = []
df_temperature_cleared = pd.DataFrame(columns = ['final_locus_population_id','event_date', 'temperature_cleared'])
for item in part_number_list:
    if item not in unic_part_number_list:
        unic_part_number_list.append(item)
n = 0
while n < len(unic_part_number_list):
    part_number = unic_part_number_list[n]
    df_temperature_filtered = df_date_temperature[(df_date_temperature['final_locus_population_id'] == part_number)]
    df_temperature_filtered.reset_index(drop = True, inplace = True)
    #df_temperanure_filtered = df_temperature_filtered.reindex(columns = df_temperature_filtered.columns.tolist()+['rolling_tempr','tempr_cleared'])
    df_temperature_filtered['rolling_tempr'] = df_temperature_filtered['temperature'].rolling(30).mean()
    df_temporary = df_temperature_filtered.head(30)
    average_tempr_30 = df_temporary['temperature'].mean()
    half_average_tempr_30 = average_tempr_30 / 2 
    k = 0
    while k < len(df_temporary):
        t_1 = df_temperature_filtered.at[k,'temperature']
        t_dif = t_1 - average_tempr_30
        if abs(t_dif) < half_average_tempr_30:
                df_temperature_filtered.at[k,'rolling_tempr'] = t_1
        else:
                df_temperature_filtered.at[k,'rolling_tempr'] = average_tempr_30
        k = k + 1        
    df_temperature_cleared = df_temperature_cleared.append(df_temperature_filtered)    
    n = n + 1
    print ('n', n)
#datatypes_1 = df_temperature_cleared.dtypes
m = 0
cc = len(df_temperature_cleared)
for m in range (cc):
    t_2 = df_temperature_cleared.iloc[m]['temperature']
    t_rolling_aver = df_temperature_cleared.iloc[m]['rolling_tempr']
    half_t_rolling_aver = t_rolling_aver / 2
    t_dif_clear = t_2 - t_rolling_aver
    if abs(t_dif_clear) < half_t_rolling_aver:
        df_temperature_cleared.at[m,'temperature_cleared'] = t_2
    else:
        df_temperature_cleared.at[m,'temperature_cleared'] = t_rolling_aver
    m = m + 1
    print ('m', m)    
   

In [ ]:
df_seawater_temperature = pd.read_csv('Salmon_project/data/seawater_temperature_for_transfers_since_2017_only_first_90days.csv')
df_seawater_temperature = df_seawater_temperature.reindex(columns = df_seawater_temperature.columns.tolist()+['key_param'])
df_sw_tempr_final = pd.DataFrame(columns = ['locus_id','transfer_date', 'to_date', 'event_date', 'average_tempr', 'min_tempr', 'max_tempr'])

n = 0
while n < len (df_seawater_temperature):
    key_param = str(df_seawater_temperature.at[n,'locus_id'])+'/'+ str(df_seawater_temperature.at[n,'transfer_date'])
    df_seawater_temperature.at[n,'key_param'] = key_param
    n = n + 1

keys_list = df_seawater_temperature['key_param'].tolist()
unic_keys_list = []
for item in keys_list:
    if item not in unic_keys_list:
        unic_keys_list.append(item)

m = 0
while m < len(unic_keys_list):
    key = unic_keys_list [m]
    df_seawater_tempr_filtered = df_seawater_temperature[(df_seawater_temperature['key_param'] == key)]
    locus_id= df_seawater_tempr_filtered.at [m,'locus_id']
    #transfer_date -
    #to_date
    #event_date
    average_tempr = df_seawater_tempr_filtered['temperature'].mean()
    #min_tempr
    #max_tempr
    m = m + 1
    print(m)